In [1]:
from flask import Flask

In [2]:
from flask import render_template

In [3]:
import pandas as pd
import numpy as np
from zipfile import ZipFile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import matplotlib.pyplot as plt

movielens_data_file_url = (
    "http://files.grouplens.org/datasets/movielens/ml-latest-small.zip"
)
movielens_zipped_file = keras.utils.get_file(
    "ml-latest-small.zip", movielens_data_file_url, extract=False
)
keras_datasets_path = Path(movielens_zipped_file).parents[0]
movielens_dir = keras_datasets_path / "ml-latest-small"

# Only extract the data the first time the script is run.
if not movielens_dir.exists():
    with ZipFile(movielens_zipped_file, "r") as zip:
        # Extract files
        print("Extracting all the files now...")
        zip.extractall(path=keras_datasets_path)
        print("Done!")

ratings_file = movielens_dir / "ratings.csv"
df = pd.read_csv(ratings_file)


983040/978202 [==============================] - 2s 2us/step
Extracting all the files now...
Done!


In [4]:
df

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [5]:
from sqlalchemy import create_engine

In [11]:
import os
import pandas as pd
from sklearn.decomposition import NMF
from fuzzywuzzy import process
import numpy as np
import googleapiclient.discovery
import google_auth_oauthlib.flow
import googleapiclient.errors

def create_engine_load_data():
    '''Load data into memory and return an engine and data as a dataframe.'''
    # Load and fill database
    engine = create_engine('sqlite:///:memory:', echo=False)

    for f in os.listdir('data/movies'):
        if f[-4:] == '.csv':
            data = pd.read_csv(f'data/movies/{f}')
            data.to_sql(f[:-4], engine)
            print(f[:-4], 'loaded succesfully')

    # LOAD DATA
    query = 'SELECT "userId", ratings."movieId", movies.title, rating FROM ratings JOIN movies ON ratings."movieId" = movies."movieId";'
    all_ratings = pd.read_sql(query, engine)

    return engine, all_ratings

In [57]:
engine = create_engine('sqlite:///data/movies/recommender.db', echo=False)

for f in os.listdir('data/movies'):
    if f[-4:] == '.csv':
        print(f[0:-4])


In [58]:
ls

app.db  data/       modeltry.py   requirements.txt  Untitled.ipynb
APP.DB  dockerfile  __pycache__/  Untitled1.ipynb   why.ipynb


In [59]:
cd data

/flask_jupyter/data


In [60]:
ls

movies/


In [61]:
cd movies

/flask_jupyter/data/movies


In [62]:
ls

ml-latest-small/


In [64]:
cd ml-latest-small/

/flask_jupyter/data/movies/ml-latest-small


In [65]:
ls

links.csv  movies.csv  RATINGS  ratings.csv  README.txt  tags.csv


In [54]:
engine = create_engine('sqlite:///data/movies/recommender.db', echo=False)


In [66]:
engine

Engine(sqlite:///data/movies/recommender.db)

In [ ]:
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///app.db'
<SQLAlchemy engine=sqlite:////flask_jupyter/app.db>

In [ ]:
table=os.path.splitext(os.path.basename(filename))[0]

                sql = 'drop table if exists "{}"'.format(table)
                db.execute(sql)

                sql = 'create table "{table}" ( {cols} )'.format(
                    table=table,
                    cols=','.join('"{}"'.format(col) for col in cols))
                db.execute(sql)

In [49]:
engine = create_engine('sqlite:///:memory:', echo=False)
engine

Engine(sqlite:///:memory:)

In [50]:
create_engine_load_data()

OperationalError: (sqlite3.OperationalError) no such table: ratings
[SQL: SELECT "userId", ratings."movieId", movies.title, rating FROM ratings JOIN movies ON ratings."movieId" = movies."movieId";]
(Background on this error at: http://sqlalche.me/e/13/e3q8)

In [38]:
pwd

'/flask_jupyter'

In [7]:
pip install fuzzywuzzy

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install googleapiclient

ERROR: Could not find a version that satisfies the requirement googleapiclient (from versions: none)
ERROR: No matching distribution found for googleapiclient
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install google-api-python-client

     |████████████████████████████████| 61 kB 35 kB/s  eta 0:00:01
     |████████████████████████████████| 91 kB 976 kB/s eta 0:00:01
     |████████████████████████████████| 95 kB 833 kB/s eta 0:00:01
     |████████████████████████████████| 100 kB 1.0 MB/s ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [40]:
create_engine_load_data()

OperationalError: (sqlite3.OperationalError) no such table: ratings
[SQL: SELECT "userId", ratings."movieId", movies.title, rating FROM ratings JOIN movies ON ratings."movieId" = movies."movieId";]
(Background on this error at: http://sqlalche.me/e/13/e3q8)

In [17]:
def process_user_input(user_input=None, all_ratings=None):
    '''Return a tuple: key(name of input field), value(user input string), df_guess(dataframe of pre-selected movie names), guesses(fuzzywuzzy of user input and df_guess as tuple(title, scoring and index))'''
    # Extract tuple
    user_input_key = user_input[0]
    user_input_value = user_input[1]
    # Pre-select movies from database. Select everything that contains the first 3 letters capitalized of user input
    df_guess = all_ratings[df['title'].str.contains(user_input_value[:3].lower().capitalize())].groupby('title').first().reset_index()
    # Get ordered list of the five most similar movie titles to user input. Return a tuple of title, match score and index.
    guesses = process.extract(user_input_value, df_guess['title'])

    return user_input_key, user_input_value, df_guess, guesses

In [18]:
process_user_input()

TypeError: 'NoneType' object is not subscriptable

In [15]:
user_ids = df["userId"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
movie_ids = df["movieId"].unique().tolist()
movie2movie_encoded = {x: i for i, x in enumerate(movie_ids)}
movie_encoded2movie = {i: x for i, x in enumerate(movie_ids)}
df["user"] = df["userId"].map(user2user_encoded)
df["movie"] = df["movieId"].map(movie2movie_encoded)

num_users = len(user2user_encoded)
num_movies = len(movie_encoded2movie)
df["rating"] = df["rating"].values.astype(np.float32)

In [16]:
df["rating"]

0         4.0
1         4.0
2         4.0
3         5.0
4         5.0
         ... 
100831    4.0
100832    5.0
100833    5.0
100834    5.0
100835    3.0
Name: rating, Length: 100836, dtype: float32

In [19]:
df["user"]

0           0
1           0
2           0
3           0
4           0
         ... 
100831    609
100832    609
100833    609
100834    609
100835    609
Name: user, Length: 100836, dtype: int64

In [20]:
pwd

'/flask_jupyter'

In [21]:
cd ..

/


In [22]:
ls

bin@   etc/            lib@    libx32@  opt/   run/   sys/  var/
boot/  flask_jupyter/  lib32@  media/   proc/  sbin@  tmp/
dev/   home/           lib64@  mnt/     root/  srv/   usr/


In [23]:
cd home

/home


In [24]:
ls

jovyan/


In [25]:
cd ../flask_jupyter

/flask_jupyter


In [26]:
ls

dockerfile  requirements.txt  Untitled1.ipynb  Untitled.ipynb  why.ipynb


In [130]:
from modeltry import raah

ImportError: cannot import name 'raah' from 'modeltry' (/flask_jupyter/modeltry.py)

In [131]:
pwd

'/flask_jupyter/data/movies/ml-latest-small'

In [132]:
cd ../../../

/flask_jupyter


In [133]:
ls

app.db  data/       modeltry.py   recommender.db    Untitled1.ipynb  why.ipynb
APP.DB  dockerfile  __pycache__/  requirements.txt  Untitled.ipynb


In [142]:
from modeltry import raaah

In [135]:
raaah()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [39]:
ls

dockerfile   __pycache__/      Untitled1.ipynb  why.ipynb
modeltry.py  requirements.txt  Untitled.ipynb


In [149]:
user_ids = df["userId"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
movie_ids = df["movieId"].unique().tolist()
movie2movie_encoded = {x: i for i, x in enumerate(movie_ids)}
movie_encoded2movie = {i: x for i, x in enumerate(movie_ids)}
df["user"] = df["userId"].map(user2user_encoded)
df["movie"] = df["movieId"].map(movie2movie_encoded)

num_users = len(user2user_encoded)
num_movies = len(movie_encoded2movie)
df["rating"] = df["rating"].values.astype(np.float32)

In [150]:
df["user"]

82065      0
16265      1
58279      2
97963      3
87033      4
        ... 
75770     60
75039     64
94466     38
80823    333
15737     67
Name: user, Length: 100836, dtype: int64

In [46]:
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)

app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///app.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SECRET_KEY'] = 'some-secret-string'

db = SQLAlchemy(app)

@app.before_first_request
def create_tables():
    db.create_all()

In [94]:
engine

Engine(sqlite:///recommender.db)

In [1]:
import os
import pandas as pd
from sklearn.decomposition import NMF
from fuzzywuzzy import process
import numpy as np
import googleapiclient.discovery
import google_auth_oauthlib.flow
import googleapiclient.errors
from sqlalchemy import create_engine

def create_engine_load_data():
    '''Load data into memory and return an engine and data as a dataframe.'''
    # Load and fill database
    engine = create_engine('sqlite:///recommender.db', echo=False)

    for f in os.listdir('data/movies/ml-latest-small'):
        if f[-4:] == '.csv':
            data = pd.read_csv(f'data/movies/ml-latest-small/{f}')
            data.to_sql(f[:-4], engine)
            print(f[:-4], 'loaded succesfully')

    # LOAD DATA
    query = 'SELECT "userId", ratings."movieId", movies.title, rating FROM ratings JOIN movies ON ratings."movieId" = movies."movieId";'
    all_ratings = pd.read_sql(query, engine)

    return engine, all_ratings

/opt/conda/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [98]:
create_engine_load_data()


links loaded succesfully
ratings loaded succesfully
movies loaded succesfully
tags loaded succesfully


(Engine(sqlite:///recommender.db),
         userId  movieId                           title  rating
 0            1        1                Toy Story (1995)     4.0
 1            1        3         Grumpier Old Men (1995)     4.0
 2            1        6                     Heat (1995)     4.0
 3            1       47     Seven (a.k.a. Se7en) (1995)     5.0
 4            1       50      Usual Suspects, The (1995)     5.0
 ...        ...      ...                             ...     ...
 100831     610   166534                    Split (2017)     4.0
 100832     610   168248   John Wick: Chapter Two (2017)     5.0
 100833     610   168250                  Get Out (2017)     5.0
 100834     610   168252                    Logan (2017)     5.0
 100835     610   170875  The Fate of the Furious (2017)     3.0
 
 [100836 rows x 4 columns])

In [112]:
engine

Engine(sqlite:///recommender.db)

In [114]:
all_ratings

,userId,movieId,title,rating
0,1,1,Toy Story (1995),4.0
1,1,3,Grumpier Old Men (1995),4.0
2,1,6,Heat (1995),4.0
3,1,47,Seven (a.k.a. Se7en) (1995),5.0
4,1,50,"Usual Suspects, The (1995)",5.0
...,...,...,...,...
100831,610,166534,Split (2017),4.0
100832,610,168248,John Wick: Chapter Two (2017),5.0
100833,610,168250,Get Out (2017),5.0
100834,610,168252,Logan (2017),5.0


In [113]:
query = 'SELECT "userId", ratings."movieId", movies.title, rating FROM ratings JOIN movies ON ratings."movieId" = movies."movieId";'
all_ratings = pd.read_sql(query, engine)

In [8]:
pwd

'/flask_jupyter'

In [9]:
ls

app.db  dockerfile              modelcopykeras.py  recommender.db    why.ipynb
APP.DB  justcopykeras.ipynb     modeltry.py        requirements.txt
data/   load_modify_data.ipynb  __pycache__/       Untitled.ipynb


In [91]:
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)

engine = create_engine('sqlite:///recommender.db', echo=False)

for f in os.listdir('data/movies/ml-latest-small'):
    if f[-4:] == '.csv':
        data = pd.read_csv(f'data/movies/ml-latest-small/{f}')
        data.to_sql(f[:-4], engine)
        print(f[0:-4])

links
ratings
movies
tags


In [92]:
#yes
data

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


In [93]:
data.to_sql(f[:-4], engine)


In [ ]:
table=os.path.splitext(os.path.basename(filename))[0]

                sql = 'drop table if exists "{}"'.format(table)
                db.execute(sql)

                sql = 'create table "{table}" ( {cols} )'.format(
                    table=table,
                    cols=','.join('"{}"'.format(col) for col in cols))
                db.execute(sql)

In [115]:
df

,userId,movieId,rating,timestamp,user,movie
0,1,1,4.0,964982703,0,0
1,1,3,4.0,964981247,0,1
2,1,6,4.0,964982224,0,2
3,1,47,5.0,964983815,0,3
4,1,50,5.0,964982931,0,4
...,...,...,...,...,...,...
100831,610,166534,4.0,1493848402,609,3120
100832,610,168248,5.0,1493850091,609,2035
100833,610,168250,5.0,1494273047,609,3121
100834,610,168252,5.0,1493846352,609,1392


In [82]:
cd ../../

/home


In [83]:
cd ..

/


In [84]:
ls

bin@   etc/            lib@    libx32@  opt/   run/   sys/  var/
boot/  flask_jupyter/  lib32@  media/   proc/  sbin@  tmp/
dev/   home/           lib64@  mnt/     root/  srv/   usr/


In [85]:
cd flask_jupyter

/flask_jupyter


In [86]:
ls

app.db  data/       modeltry.py   requirements.txt  Untitled.ipynb
APP.DB  dockerfile  __pycache__/  Untitled1.ipynb   why.ipynb


In [73]:
ls

bin@   etc/            lib@    libx32@  opt/   run/   sys/  var/
boot/  flask_jupyter/  lib32@  media/   proc/  sbin@  tmp/
dev/   home/           lib64@  mnt/     root/  srv/   usr/


In [74]:
cd ../

/


In [75]:
ls

bin@   etc/            lib@    libx32@  opt/   run/   sys/  var/
boot/  flask_jupyter/  lib32@  media/   proc/  sbin@  tmp/
dev/   home/           lib64@  mnt/     root/  srv/   usr/


In [47]:
create_tables()

In [48]:
db

<SQLAlchemy engine=sqlite:////flask_jupyter/app.db>

In [44]:
pip install flask_sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [107]:
ratings = pd.read_csv('ratings.csv')

In [104]:
ls

app.db  data/       modeltry.py   recommender.db    Untitled1.ipynb  why.ipynb
APP.DB  dockerfile  __pycache__/  requirements.txt  Untitled.ipynb


In [105]:
cd data/movies/ml-latest-small

/flask_jupyter/data/movies/ml-latest-small


In [106]:
ls

links.csv  movies.csv  RATINGS  ratings.csv  README.txt  tags.csv


In [108]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [110]:
df['rating']

0         4.0
1         4.0
2         4.0
3         5.0
4         5.0
         ... 
100831    4.0
100832    5.0
100833    5.0
100834    5.0
100835    3.0
Name: rating, Length: 100836, dtype: float32

In [124]:
df = df.sample(frac=1, random_state=42)
x = df[["user", "movie"]].values
# Normalize the targets between 0 and 1. Makes it easy to train.
y = df["rating"].apply(lambda x: (x - min_rating) / (max_rating - min_rating)).values
# Assuming training on 90% of the data and validating on 10%.
train_indices = int(0.9 * df.shape[0])
x_train, x_val, y_train, y_val = (
    x[:train_indices],
    x[train_indices:],
    y[:train_indices],
    y[train_indices:],
)


In [ ]:
x_train, x_val, y_train, y_val = (


In [117]:
min_rating = min(df["rating"])


In [ ]:
max_rating = max(df["rating"])

In [125]:
min_rating

0.5

In [126]:
max_rating

5.0

In [121]:
df["movie"]

39920    6209
31097     355
27388    1470
35836      20
12082     209
         ... 
99583    1025
91864    1117
59035      27
25661    5267
97176      47
Name: movie, Length: 100836, dtype: int64

In [122]:
df["user"]

39920    273
31097    216
27388    185
35836    242
12082     73
        ... 
99583    609
91864    595
59035    384
25661    176
97176    604
Name: user, Length: 100836, dtype: int64

In [146]:
df["rating"]

82065    4.0
16265    4.0
58279    3.5
97963    2.5
87033    4.0
        ... 
75770    4.0
75039    4.0
94466    2.5
80823    3.5
15737    4.0
Name: rating, Length: 100836, dtype: float32

In [123]:
df = df.sample(frac=1, random_state=42)
df

,userId,movieId,rating,timestamp,user,movie
66674,429,195,4.0,828124616,428,543
79937,500,1148,4.0,1005528205,499,1099
98086,606,5614,4.5,1171404629,605,9463
53636,354,1214,4.0,1200870323,353,75
42812,288,3672,3.0,978468053,287,6001
...,...,...,...,...,...,...
6783,45,3489,3.0,959625135,44,217
99491,608,51662,5.0,1189563889,607,899
59974,387,6440,3.5,1194158404,386,1210
31410,217,2986,3.0,955941623,216,5128


In [ ]:
user_ids = df["userId"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
movie_ids = df["movieId"].unique().tolist()
movie2movie_encoded = {x: i for i, x in enumerate(movie_ids)}
movie_encoded2movie = {i: x for i, x in enumerate(movie_ids)}
df["user"] = df["userId"].map(user2user_encoded)
df["movie"] = df["movieId"].map(movie2movie_encoded)

num_users = len(user2user_encoded)
num_movies = len(movie_encoded2movie)
df["rating"] = df["rating"].values.astype(np.float32)
# min and max ratings will be used to normalize the ratings later
min_rating = min(df["rating"])
max_rating = max(df["rating"])

print(
    "Number of users: {}, Number of Movies: {}, Min rating: {}, Max rating: {}".format(
        num_users, num_movies, min_rating, max_rating

In [127]:
user_movie_ratings = pd.pivot_table(all_ratings, values='rating', index='userId', columns='movieId')
user_movie_ratings

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,2.5,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
user_movie_ratings

NameError: name 'user_movie_ratings' is not defined

In [5]:
df

NameError: name 'df' is not defined